In [10]:
# using this notebook to create feature data from AlexNet, inception, and other famous models --> going to create
# an ensemble method of sorts

from keras.applications.resnet50 import ResNet50
#from keras.applications.inception_resnet_v2 import InceptionResNetV2
#from keras.applications.inception_v3 import InceptionV3
    #inception_v3_feature_data = np.zeros([14, 5564, 1634304])
    
import cv2
from keras.models import Model
import numpy as np
import os


In [11]:
model = ResNet50(include_top=False, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [12]:
# function for padding each input image
def pad(array, reference_shape, offsets):
    """
    array: Array to be padded
    reference_shape: tuple of size of ndarray to create
    offsets: list of offsets (number of elements must be equal to the dimension of the array)
    will throw a ValueError if offsets is too big and the reference_shape cannot handle the offsets
    """

    # Create an array of zeros with the reference shape
    result = np.zeros(reference_shape)
    # Create a list of slices from offset to offset + shape in each dimension
    insertHere = [slice(offsets[dim], offsets[dim] + array.shape[dim]) for dim in range(array.ndim)]
    # Insert the array in the result at the specified offsets
    result[insertHere] = array
    return result

In [ ]:
# extracting feature data for all of the movies
# iterate though each movie in the folder
num_movies = 14
max_width = 1280
max_height = 720

# set up master array
resnet50_feature_data = np.zeros([14, 5564, 30720])

for num in range(num_movies):
    count = 0
    mov = num 
    for file in os.listdir('../Desktop/frames/{}/'.format(mov)):
        filename = os.fsdecode(file)
        if filename.endswith(".jpg"):
            # extract the image data
            test_img = cv2.imread(os.path.join('../Desktop/frames/{}/'.format(mov), filename))
            # pad the image to that it is of max width and max height
            test_img = pad(test_img,(max_width, max_height,3), (0,0,0)) # no offset
            data = (test_img/255)[np.newaxis, :, :, :]
            # send image data through the resnet model
            intermediate_output = model.predict(data)
            # reshape model output
            feature_data = intermediate_output.flatten()
            #print(feature_data.shape)
            # slot into the master away
            resnet50_feature_data[mov, count, :feature_data.shape[0]] = feature_data
        
            count = count + 1
            continue
        else:
            continue
    print(mov)

0
1
2
